In [1]:
import os 
import numpy as np
import pandas as pd

import lightgbm as lgb

import keras 
from keras.layers import *
from keras.optimizers import *
from keras.models import Model

from sklearn.metrics import f1_score 
from sklearn.model_selection import KFold

2023-04-21 14:49:20.046765: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-21 14:49:20.080132: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-21 14:49:20.080764: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-21 14:49:20.864401: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
DIR  = "567/"
SEED = 1881

if not os.path.isdir("models/"):
    os.makedirs("models")
    
print(os.listdir(DIR))

['567auto.ipynb', 'train_values.csv', 'Untitled.ipynb', '.ipynb_checkpoints', 'smallgroup.ipynb', 'Catboost.ipynb', 'AutogluonModels', 'Untitled2.ipynb', 'train_labels.csv', 'Untitled1.ipynb', 'submission.csv', 'test_values.csv', 'submission_format.csv']


In [52]:
train_x = pd.read_csv(DIR+"train_values.csv")
train_y = pd.read_csv(DIR+"train_labels.csv")
test_x  = pd.read_csv(DIR+"test_values.csv")
sub_csv = pd.read_csv(DIR+"submission_format.csv")

In [53]:
geo1 = np.array(pd.get_dummies(pd.concat([train_x["geo_level_1_id"], test_x["geo_level_1_id"]])))
geo2 = np.array(pd.get_dummies(pd.concat([train_x["geo_level_2_id"], test_x["geo_level_2_id"]])))
geo3 = np.array(pd.get_dummies(pd.concat([train_x["geo_level_3_id"], test_x["geo_level_3_id"]])))

In [54]:
geo3.shape

(347469, 11861)

In [55]:
def NET():
    inp = Input((geo3.shape[1],))
    i1 = Dense(16, name="intermediate")(inp)
    x2 = Dense(geo2.shape[1], activation='sigmoid')(i1)
    x1 = Dense(geo1.shape[1], activation='sigmoid')(i1)

    model = Model(inp, [x2,x1])
    model.compile(loss="binary_crossentropy", optimizer="adam")
    return model

In [56]:
model = NET()
model.fit(geo3, [geo2, geo1], batch_size=128, epochs=10, verbose=2)
model.save("geo_embed.h5")

Epoch 1/10
2715/2715 - 13s - loss: 0.2047 - dense_12_loss: 0.0531 - dense_13_loss: 0.1516 - 13s/epoch - 5ms/step
Epoch 2/10
2715/2715 - 12s - loss: 0.0529 - dense_12_loss: 0.0052 - dense_13_loss: 0.0478 - 12s/epoch - 4ms/step
Epoch 3/10
2715/2715 - 10s - loss: 0.0145 - dense_12_loss: 0.0041 - dense_13_loss: 0.0103 - 10s/epoch - 4ms/step
Epoch 4/10
2715/2715 - 11s - loss: 0.0066 - dense_12_loss: 0.0035 - dense_13_loss: 0.0031 - 11s/epoch - 4ms/step
Epoch 5/10
2715/2715 - 11s - loss: 0.0043 - dense_12_loss: 0.0030 - dense_13_loss: 0.0013 - 11s/epoch - 4ms/step
Epoch 6/10
2715/2715 - 13s - loss: 0.0029 - dense_12_loss: 0.0024 - dense_13_loss: 5.8481e-04 - 13s/epoch - 5ms/step
Epoch 7/10
2715/2715 - 12s - loss: 0.0020 - dense_12_loss: 0.0017 - dense_13_loss: 3.0124e-04 - 12s/epoch - 4ms/step
Epoch 8/10
2715/2715 - 11s - loss: 0.0013 - dense_12_loss: 0.0011 - dense_13_loss: 1.5782e-04 - 11s/epoch - 4ms/step
Epoch 9/10
2715/2715 - 11s - loss: 8.2438e-04 - dense_12_loss: 7.3996e-04 - dense_13

In [57]:
# Load GEO-Embed Model
model = NET()
model.load_weights("geo_embed.h5")
# "Extract Intermediate Layer" Function
from keras import backend as K

get_int_layer_output = K.function([model.layers[0].input],
                                  [model.layers[1].output])


In [58]:
# Extract GEO-Embeds for all test data points.
# Then assign with test_data

out = []
for dat in geo3[:260601]:
    layer_output = get_int_layer_output([np.array(dat).reshape(1, -1)])[0]
    out.append(layer_output)

out = np.array(out)
out = np.squeeze(out)

In [59]:
train_data = pd.get_dummies(train_x.copy())
train_data = train_data.drop(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id'], axis=1)
train_data = train_data.assign(geo_feat1=out[:,0],
                               geo_feat2=out[:,1],
                               geo_feat3=out[:,2],  
                               geo_feat4=out[:,3],
                               geo_feat5=out[:,4],    
                               geo_feat6=out[:,5],
                               geo_feat7=out[:,6],
                               geo_feat8=out[:,7],
                               geo_feat9=out[:,8],
                               geo_feat10=out[:,9],
                               geo_feat11=out[:,10],
                               geo_feat12=out[:,11],
                               geo_feat13=out[:,12],
                               geo_feat14=out[:,13],
                               geo_feat15=out[:,14],           
                               geo_feat16=out[:,15])

In [60]:
# Extract GEO-Embeds for all test data points.
# Then assign with test_data

out = []
for dat in geo3[260601:]:
    layer_output = get_int_layer_output([np.array(dat).reshape(1, -1)])[0]
    out.append(layer_output)

out = np.array(out)
out = np.squeeze(out)

test_data = pd.get_dummies(test_x.copy())
test_data = test_data.drop(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id'], axis=1)
test_data = test_data.assign(geo_feat1=out[:,0],
                             geo_feat2=out[:,1],
                             geo_feat3=out[:,2],  
                             geo_feat4=out[:,3],
                             geo_feat5=out[:,4],    
                             geo_feat6=out[:,5],
                             geo_feat7=out[:,6],
                             geo_feat8=out[:,7],
                             geo_feat9=out[:,8],
                             geo_feat10=out[:,9],
                             geo_feat11=out[:,10],
                             geo_feat12=out[:,11],
                             geo_feat13=out[:,12],
                             geo_feat14=out[:,13],
                             geo_feat15=out[:,14],           
                             geo_feat16=out[:,15])

In [61]:
def threshold_arr(array):
    # Get major confidence-scored predicted value.
    new_arr = []
    for ix, val in enumerate(array):
        loc = np.array(val).argmax(axis=0)
        k = list(np.zeros((len(val))))
        k[loc]=1
        new_arr.append(k)
        
    return np.array(new_arr)

In [62]:
from numpy.lib.function_base import delete
import pandas as pd
y = np.array(train_y["damage_grade"])-1
SEED = 1881
df = train_data.drop(["building_id"], axis=1)
x = np.array(df)

In [63]:
kf = KFold(n_splits=10, shuffle=True, random_state=SEED)
for ix, (train_index, test_index) in enumerate(kf.split(x)):
    lgb_params = {
        "objective" : "multiclass",
        "num_class":3,
        "metric" : "multi_error",
        "boosting": 'gbdt',
        "max_depth" : -1,
        "num_leaves" : 33,
        "learning_rate" : 0.05,
        "feature_fraction" : 0.4,
        "min_sum_hessian_in_leaf" : 0.1,
        "max_bin":8192,
        "verbosity" : 1,
        "num_threads":6,
        "seed": SEED
    }

    x_train, x_val, y_train, y_val= x[train_index], x[test_index], y[train_index], y[test_index]

    train_data = lgb.Dataset(x_train, label=y_train)
    val_data   = lgb.Dataset(x_val, label=y_val)

    lgb_clf = lgb.train(lgb_params,
                        train_data,
                        20000,
                        valid_sets = [val_data],
                        early_stopping_rounds=3000,
                        verbose_eval = 1000)

    y_pred = lgb_clf.predict(x_val)
    print("F1-MICRO SCORE: ", f1_score(np.array(pd.get_dummies(y_val)), threshold_arr(y_pred), average='micro'))
    lgb_clf.save_model(f'models/model{ix}.txt')

/home1/ywang003/.local/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home1/ywang003/.local/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.057212 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 121527
[LightGBM] [Info] Number of data points in the train set: 234540, number of used features: 81
[LightGBM] [Info] Start training from score -2.334953
[LightGBM] [Info] Start training from score -0.564079
[LightGBM] [Info] Start training from score -1.095713
Training until validation scores don't improve for 3000 rounds
[1000]	valid_0's multi_error: 0.247688
[2000]	valid_0's multi_error: 0.245578
[3000]	valid_0's multi_error: 0.245232
[4000]	valid_0's multi_error: 0.245232
[5000]	valid_0's multi_error: 0.245616
Early stopping, best iteration is:
[2718]	valid_0's multi_error: 0.244273
F1-MICRO SCORE:  0.755726948313572


/home1/ywang003/.local/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home1/ywang003/.local/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.083935 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 121454
[LightGBM] [Info] Number of data points in the train set: 234541, number of used features: 81
[LightGBM] [Info] Start training from score -2.338398
[LightGBM] [Info] Start training from score -0.564458
[LightGBM] [Info] Start training from score -1.094073
Training until validation scores don't improve for 3000 rounds
[1000]	valid_0's multi_error: 0.246201
[2000]	valid_0's multi_error: 0.244743
[3000]	valid_0's multi_error: 0.243131
[4000]	valid_0's multi_error: 0.243246
[5000]	valid_0's multi_error: 0.244589
[6000]	valid_0's multi_error: 0.245587
Early stopping, best iteration is:
[3425]	valid_0's multi_error: 0.242556
F1-MICRO SCORE:  0.7574443591711435


/home1/ywang003/.local/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home1/ywang003/.local/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


...

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.099718 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 121429
[LightGBM] [Info] Number of data points in the train set: 234541, number of used features: 80
[LightGBM] [Info] Start training from score -2.340123
[LightGBM] [Info] Start training from score -0.563581
[LightGBM] [Info] Start training from score -1.095067
Training until validation scores don't improve for 3000 rounds
[1000]	valid_0's multi_error: 0.254566
[2000]	valid_0's multi_error: 0.249117
[3000]	valid_0's multi_error: 0.247966
[4000]	valid_0's multi_error: 0.249885
[5000]	valid_0's multi_error: 0.249309
[6000]	valid_0's multi_error: 0.249117
Early stopping, best iteration is:
[3502]	valid_0's multi_error: 0.247544
F1-MICRO SCORE:  0.752455871066769


/home1/ywang003/.local/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home1/ywang003/.local/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.058978 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 121437
[LightGBM] [Info] Number of data points in the train set: 234541, number of used features: 80
[LightGBM] [Info] Start training from score -2.343715
[LightGBM] [Info] Start training from score -0.563859
[LightGBM] [Info] Start training from score -1.093564
Training until validation scores don't improve for 3000 rounds
[1000]	valid_0's multi_error: 0.249117
[2000]	valid_0's multi_error: 0.245357
[3000]	valid_0's multi_error: 0.242594
[4000]	valid_0's multi_error: 0.243208
[5000]	valid_0's multi_error: 0.243975
Early stopping, best iteration is:
[2984]	valid_0's multi_error: 0.242364
F1-MICRO SCORE:  0.7576362240982348


/home1/ywang003/.local/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home1/ywang003/.local/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.104647 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 121433
[LightGBM] [Info] Number of data points in the train set: 234541, number of used features: 80
[LightGBM] [Info] Start training from score -2.337647
[LightGBM] [Info] Start training from score -0.564023
[LightGBM] [Info] Start training from score -1.095029
Training until validation scores don't improve for 3000 rounds
[1000]	valid_0's multi_error: 0.248503
[2000]	valid_0's multi_error: 0.245472
[3000]	valid_0's multi_error: 0.24482
[4000]	valid_0's multi_error: 0.245395
[5000]	valid_0's multi_error: 0.245127
[6000]	valid_0's multi_error: 0.245434
Early stopping, best iteration is:
[3291]	valid_0's multi_error: 0.243668
F1-MICRO SCORE:  0.7563315425940138


In [64]:
# Load all LightGB Models and concatenate.
models = []
for i in range(10):
    model = lgb.Booster(model_file=f'models/model{i}.txt')

    y_pred = model.predict(x)
    score  = f1_score(np.array(pd.get_dummies(y)), threshold_arr(y_pred), average='micro')
    print("F1-MICRO SCORE: ", score)
    models.append(model)

F1-MICRO SCORE:  0.8081741819870223
F1-MICRO SCORE:  0.8178057643677499
F1-MICRO SCORE:  0.7991412158817502
F1-MICRO SCORE:  0.8285885318935844
F1-MICRO SCORE:  0.829018307681091
F1-MICRO SCORE:  0.8302462385025383
F1-MICRO SCORE:  0.8049124907425527
F1-MICRO SCORE:  0.8194174235708994
F1-MICRO SCORE:  0.8118541371675472
F1-MICRO SCORE:  0.8162900372600259


In [65]:
def ensemble(models, x):
    # Ensemble K-Fold CV models with adding all confidence score by class.
    y_preds = []
    
    for model in models:
        y_pred = model.predict(x)
        y_preds.append(y_pred)
        
    init_y_pred = y_preds[0]
    for ypred in y_preds[1:]:
        init_y_pred += ypred
        
    y_pred = threshold_arr(init_y_pred)
    
    return y_pred

In [66]:
df = test_data.drop(["building_id"], axis=1)
x = np.array(df)
y_pred = ensemble(models, x)
y_pred_label = y_pred.argmax(axis=1)+1

In [67]:
submission_format = pd.read_csv(DIR+'submission_format.csv', index_col='building_id')
my_submission = pd.DataFrame(data=y_pred_label,
                             columns=submission_format.columns,
                             index=submission_format.index)
my_submission.to_csv('submission_final.csv')